## The following functions make up the Automated Layer Analysis tool. Individually, functions extract information from image and segmented nuclei data to assess layer properties as described in the read-me. 

In [17]:
import os                                   
import numpy as np                            
import skimage.io as io            
from scipy import optimize, stats                      
import pandas as pd                             
import math
import inspect
import time
from scipy import signal
from IPython.display import clear_output
import matplotlib.pyplot as plt                   
%matplotlib notebook

#### This first cell is intended to read in your data set as quickly and hassle free as possible.

In [18]:
toc = time.time()

#import and read raw data as data files and images from designated directory

path = "your path"
files = os.listdir(path) #returns list of files in directory
#Sorts excel files and images into their own lists
excel_files = [f for f in files if f[-4:] == 'xlsx']
images = [f for f in files if f[-3:] == 'tif']
    
#Makes a separate list of the data files with each variable name converted to a string (list of strings)
#Additionally, each variable has the ".tif" removed from its end
list_of_image_names = [sub[:-4] for sub in images]

#Sorts lists of excel and image files in the same manner/method dicatated by the sort function.
#Now, the two lists should line up excatly, i.e. first element of each correspond to the same test/well,
# howoever, due to unpaired images/sheets (e.g and excel file with no corresponding image) 
# we must first remove said unpaired elements before the two lists will align
excel_files.sort()
images.sort()

#Want to eliminate elements which do not have a corresponding image/excel sheet.
#Already have list_of_image_names — can also make list_of_excel_names then use splicing to cross reference.
list_of_excel_names = [sub[:-5] for sub in excel_files] #cuts ".xlsx" off the end of the list of excel files

#Defines two list which consist of elements not present in both lists
unmatched_excels = [i for i in list_of_excel_names if i[:] not in list_of_image_names]
unmatched_images = [i for i in list_of_image_names if i[:] not in list_of_excel_names]

#Removes the unmatched excel list elements, adds the ".xlsx" suffix onto each element of the list, and then defines the
# list with the suffix as a new, aligned list
if len(unmatched_excels) > 0:  
    for i in range(len(unmatched_excels)):
        list_of_excel_names.remove(unmatched_excels[i]) 
excel_suffix = '.xlsx'
aligned_excel_files = [sub + excel_suffix for sub in list_of_excel_names]
aligned_excel_files.sort()
#Removes the unmatched image list elements, adds the ".tif" suffix onto each element of the list, and then defines the
# list with the suffix as a new, aligned list
if len(unmatched_images) > 0:
    for i in range(len(unmatched_images)):
        list_of_image_names.remove(unmatched_images[i])
image_suffix = '.tif'
aligned_images = [sub + image_suffix for sub in list_of_image_names]
aligned_images.sort()
#Makes a list of the excel files read through as data files (list of variables)
list_of_dfs = []
for item in aligned_excel_files:
    list_of_dfs.append(pd.read_excel(path + item))
    
#Makes a list of read-in images which still need to be systemized using image shuffle
list_of_unshuffled_images = []

for item in aligned_images:
    list_of_unshuffled_images.append(io.imread(path + item))

tic = time.time()
print(tic-toc)

0.6281142234802246


In [25]:
list_of_images = []
for image in list_of_unshuffled_images:
    list_of_images.append(image_shuffle(image))

#### The following functions are detailed in the read me, along with example uses. 

In [24]:
toc = time.time()

### image shuffle ensures all of the images have the channels and xyz coordinates in the same order
### and makes a new list of images so the following definitions work correctly

#Since we know that x and y must each be 512 in length for our images, each logic statement starts by assigning
#the x and y axis to those variables (a,b,c,d) which have value 512. Then, we use the criteria that there can only
#be 4 colors maximum in our image. Therefore, after assigning the x and y axis, a variable with value 
#greater than 4 must represent the z-axis, and the remainder the color.
## In order to use this routine for an image of another size, one only has to change the length (512) values within the 
## routine to the desired length. Otherwise, this program will not work as intended.
def image_shuffle(image):
    a, b, c, d = image.shape
    if a == 512:
        xloc = 0
        if b == 512:
            yloc = 1
            if c >4:
                zloc = 2
                colloc = 3
            else:
                zloc = 3
                colloc = 2
        elif c == 512:
            yloc = 2
            if b > 4:
                zloc = 1
                colloc = 3
            else:
                zloc = 3
                colloc = 1
        else:
            yloc = 3
            if b > 4:
                zloc = 1
                colloc = 2
            else:
                zloc = 2
                colloc = 1
    elif b == 512:
        xloc = 1
        if c == 512:
            yloc = 2
            if a > 4:
                zloc = 0
                colloc = 3
            else:
                zloc = 3
                colloc = 0
        else:
            yloc = 3
            if a > 4:
                zloc = 0
                colloc = 2
            else:
                zloc = 2
                colloc = 0
    else:
        xloc = 2
        yloc = 3
        if a > 4:
            zloc = 0
            colloc = 1
        else:
            zloc = 1
            colloc = 0
    return np.transpose(image, (zloc, colloc, xloc, yloc))

### Get_layer_position extracts the x, y, z min and max positions from the microscope, returns them, and 
### also defines an array of equal value slices which add up to form the entire layer.
    
def get_layer_position(df, image):
    num_z, num_c, num_x, num_y = image.shape
    df.columns = ['x', 'y', 'z', 'vol', 'positions']
    x_max = df['positions'][0]
    y_max = df['positions'][1]
    z_max = df['positions'][2]
    x_min = df['positions'][3]
    y_min = df['positions'][4]
    z_min = df['positions'][5]
    slice_heights = np.linspace(0, z_max-z_min, num = num_z)
    return (x_max, y_max, z_max, x_min, y_min, z_min, slice_heights)

### local density allows a user to define a smaller area (a box) to analyze if desired. The default is the maximum size
### of the image provided
def local_density(df, image, **kwargs):
    x_max, y_max, z_max, x_min, y_min, z_min, slice_heights = get_layer_position(df, image)
    box_radius = kwargs.get('box_radius', (x_max-x_min)/2) #assigning radius value according to desired box size
    df_cleared = clear_debris(df) #clearing out background noise
    xcent= (xmin+xmax)/2 
    ycent= (ymin+ymax)/2 #defining centers of the x and y planes of layer
    xmin_box, xmax_box = xcent - box_radius, xcent + box_radius
    ymin_box, ymax_box = ycent- box_radius, ycent + box_radius #defining bounds in x & y for the chosen box
    df_box = df_cleared[(df_cleared['x'].values >= xmin_box)& 
                        (df_cleared['x'].values <= xmax_box)&
                        (df_cleared['y'].values >= ymin_box)&
                        (df_cleared['y'].values <= ymax_box)] #Clearing only those cells within the given box parameters
    number_of_cells_in_box = len(df_box) #number of cells in box is the same as the number cleared in the previous line
    box_density = (number_of_cells_in_box / (4*box_radius**2))*1000
    return number_of_cells_in_box, box_density


### clear debris removes any volumes in the provided segmentation file to ensure that only nuclei are being 
### included in the following analysis

#Removes false nuclei/background noise based upon the volume of the sources. 
def clear_debris(df, **kwargs):
    df.columns = ['x', 'y', 'z', 'vol', 'positions']
    plot = kwargs.get('plot', False)
    save_name = kwargs.get('save_name', False)
    vols = df['vol'].values 
    bottom_vol = np.mean(vols)- (1.5*np.std(vols)) #cutoff volume
    df_cleared = df[df['vol']>=bottom_vol] #all sources below the cutoff volume are excluded
    if plot == True: #If plot is passed as an argument, the following plotting sequence will run.
        bins = np.linspace(np.min(vols), np.max(vols), num = 100)
        fig, ax = plt.subplots()
        ax.hist(vols, bins = bins, alpha = 1, label= 'Debris')
        ax.hist(df_cleared['vol'].values, bins = bins, alpha = 1, label = 'Cells')
        ax.legend(loc = 'upper right')
        ax.set_ylabel('Number of cells')
        ax.set_xlabel('Nucleus volume ($\mu$m$^3$)')
        ax.axvline(x = bottom_vol, c='k', linestyle = '--')
        fig.show
        if save_name != False: #If a save_name is passed as an argument, the figure will be saved. This idea holds for all 
                                #future routines where this is used.
            plt.savefig(save_name + '.pdf', bbox_inches = 'tight', pad_inches = 1)
    return(df_cleared)

### layer height actin uses the xy projection of the image in order 
#Uses the normalized actin intensity graph to determine z positions for the start/end of a given layer. Also 
#determines layer height and the actin peak location
def layer_height_actin(df, image, **kwargs):
    x_max, y_max, z_max, x_min, y_min, z_min, slice_heights = get_layer_position(df, image)
    plot = kwargs.get('plot', False)
    save_name = kwargs.get('save_name', False)
    actin_channel = kwargs.get('actin_channel', 1)
    
    
    xy_proj_actin = np.sum(np.sum(image, axis = 3), axis = 2)[:,actin_channel].copy() #measuring the xy projecting of the actin intensity
                                                                                      # x and y were assigned to axis 2 and 3 in image shuffle
    norm_intensities_actin = (xy_proj_actin-np.min(xy_proj_actin))/np.max(xy_proj_actin-np.min(xy_proj_actin)) #normalize actin graph
    
    actin_peak = slice_heights[np.argwhere(norm_intensities_actin == np.max(norm_intensities_actin))][0][0] #determine actin peak z-value
    df_cleared = clear_debris(df)
    density = len(df_cleared)/(x_max-x_min)**2*1000
    if density <=2:
        bot_cutoff = 0.5
        top_cutoff = 0.6
    elif density>=6:
        bot_cutoff = 0.2
        top_cutoff = 0.8
    else:
        bot_cutoff = 0.5-0.3*(density-2)/4
        top_cutoff = 0.6+0.2*(density-2)/4
    print(density, bot_cutoff, top_cutoff)
    min_actin_slice = np.argwhere(norm_intensities_actin >= bot_cutoff)[0][0]
    max_actin_slice = np.argwhere(norm_intensities_actin >= top_cutoff)[-1][0] #Determining locations of bottom and top of layer using empirical results
    min_layer_height = slice_heights[min_actin_slice]
    max_layer_height = slice_heights[max_actin_slice] #Determining values of layer height at bottom and top bounds found above
    layer_height = max_layer_height-min_layer_height
    actin_intensity_top = norm_intensities_actin[max_actin_slice] #actin intensity value at top of layer
    
    if (plot == True):
        fig, ax = plt.subplots()
        ax.plot(slice_heights, norm_intensities_actin, c='cornflowerblue')
        ax.set_ylabel('Actin Intensity (A.U.)')
        ax.set_xlabel('Z-Position ($\mu$m)')
        ax.axvline(x = min_layer_height, c='k', linestyle = '--', label = 'Layer Bounds')
        ax.axvline(x = max_layer_height, c = 'k', linestyle = '--')
        ax.axvline(x = np.max(slice_heights), c = 'r', linestyle = '--')
        ax.legend(loc='upper right')
        ax.set_ylim([0,1.1])
        ax.set_xlim([0, 30])
        if save_name != False:
            plt.savefig(save_name + '.pdf', bbox_inches = 'tight', pad_inches = 1)
    return (min_layer_height, max_layer_height, layer_height, actin_peak, norm_intensities_actin)

#Smooths actin intensity graph so that it can be differentiated nicely. This is done so that when we use the derivative
# in the next routine, find_shoulders, there are no problems due to the jagged areas of original actin intensity graph.
## One thing to note is that the smoothed graphs for both the mature and immature cases are normalized, so they appear
## to have the same intensity even though the mature case has a higher absolute intensity.
def smooth_array(array, **kwargs):
    number_to_smooth = kwargs.get('window_size', 3)
    window = np.ones(number_to_smooth)/number_to_smooth #array to use in convolution
    new_array = np.convolve(array, window, mode='same') #Convoluting a given array in order to smooth sharp edges
    return new_array

#We can differentiate between mature and immature based upon the presence of a secondary, "shoulder" on the actin 
#intensity graph. Mature cases will have the shoulder which is a result of the increased presence of lateral (cell to 
#cell) actin relative to immature cases. The following routine detects the presence of shoulders which have a prominence
#above a certain degree. It does this using the derivative and a given prominence value.
def find_shoulders(df, image, **kwargs):
    x_max, y_max, z_max, x_min, y_min, z_min, slice_heights = get_layer_position(df, image)
    plot = kwargs.get('plot', False)
    save_name = kwargs.get('save_name', False)
    actin_channel  = kwargs.get('actin_channel', 1)
    xy_proj_actin = np.sum(np.sum(image, axis = 3), axis = 2)[:,actin_channel].copy()
    norm_intensities_actin = (xy_proj_actin-np.min(xy_proj_actin))/np.max(xy_proj_actin-np.min(xy_proj_actin))
    actin_profile_derivative = smooth_array(np.diff(norm_intensities_actin), window_size = 5)
    peaks = signal.find_peaks(actin_profile_derivative, prominence = 0.008)
    
    if len(peaks[0])==2:
        equivalence = peaks[1]['prominences'][1]/peaks[1]['prominences'][0]
    else:
        equivalence = 'undef'
    
    if plot == True or save_name != False:
        fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (7,7))
        ax.plot(slice_heights, norm_intensities_actin, 'cornflowerblue', label = 'Actin Profile')
        ax.set_ylabel('Actin Intensity (A.U.)')
        ax.set_xlabel('Z-Position ($\mu$m)')
        ax2=ax.twinx()
        ax2.plot(slice_heights[:-1] + (z_max-z_min)/(2*len(slice_heights)), actin_profile_derivative, 'steelblue', label = 'Actin Derivative')
        ax2.set_xlabel('Z-Position ($\mu$m)')
        ax2.set_ylabel('Actin Profile 1$^{st}$ Derivative (A.U. / $\mu$m)')
        fig.tight_layout()
        fig.show()
        if save_name != False:
            plt.savefig(save_name + '.pdf', bbox_inches = 'tight', pad_inches = 1)
    
    return (equivalence, len(peaks[0]), peaks[1]['prominences'][1], peaks[1]['prominences'][0])

def gaussian_fit(x, A, B, C):
    return A * np.exp(-1 * ((x - B) / C) ** 2)

def double_gaussian_fit(x, A, B, C, D, E, F):
    return A * np.exp(-1 * ((x - B) / C) ** 2) + D * np.exp(-1 * ((x - E) / F) ** 2)

#Classifies layers as organized, disorganized, or undefined as well as finds the location of the nuclear peak
def nuclei_distribution(df, image, **kwargs):
    x_max, y_max, z_max, x_min, y_min, z_min, slice_heights = get_layer_position(df, image)
    min_layer_height, max_layer_height, layer_height, actin_peak, norm_intensities_actin = layer_height_actin(df, image)
    plot = kwargs.get('plot', False)
    save_name = kwargs.get('save_name', False)
    df_cleared = clear_debris(df)
    zs = df_cleared['z'].values - z_min
    bins = np.linspace(0, 50, 51)
    bins_fit = np.linspace(0, 50, 1000)
    counts, bins_aux = np.histogram(zs, bins = bins)
    counts_to_fit = smooth_array(counts)
    p0_double = [np.max(counts_to_fit), 8, 2, 30, 15, 5]
    p0_single = [np.max(counts_to_fit), 8, 2]
    bounds_double = ([0, 1, 1, 0, 1, 1], [400, 80, 15, 400, 80, 15])
    bounds_single = ([0, 1, 1], [400, 80, 15])
    peak_loc = np.argwhere(counts_to_fit==np.max(counts_to_fit))[0][0]
    params_double, params_covariance_double = optimize.curve_fit(double_gaussian_fit, bins[:-1], counts_to_fit, p0_double, bounds = bounds_double)
    params_single, params_covariance_single = optimize.curve_fit(gaussian_fit, bins[:-1], counts_to_fit, p0_single, bounds = bounds_single)
    fit_single = gaussian_fit(bins_fit, params_single[0], params_single[1], params_single[2])
    fit_double = double_gaussian_fit(bins_fit, params_double[0], params_double[1], params_double[2], params_double[3], params_double[4], params_double[5])
    
    mean_deviation = np.sqrt(np.mean((fit_single - fit_double)**2))
    
    if mean_deviation >5:
        peaks = 2
        if params_double[0]>=params_double[3]:
            tall_peak = params_double[:3]
            short_peak = params_double[3:]
        else:
            tall_peak = params_double[3:] 
            short_peak = params_double[:3]
        
        if params_double[1]<=params_double[4]:
            left_peak = params_double[:3]
            right_peak = params_double[3:]
        else:
            left_peak = params_double[3:] 
            right_peak = params_double[:3]
        
        a1, b1, c1 = left_peak
        a2, b2, c2 = right_peak
    
        A = 1/c2**2-1/c1**2
        B = 2*b1/c1**2-2*b2/c2**2
        C = (b2/c2)**2-(b1/c1)**2+np.log(a1/a2)
    
        roots = np.roots([A,B,C])
        intersection = np.max(gaussian_fit(roots, a1,b1,c1))
        same_spot_value = intersection / short_peak[0]

        if same_spot_value <= 0.5:
            layer = 'organized'
            nuclear_peak = left_peak[1]
        else:
            layer = 'disorganized'
            nuclear_peak = left_peak[1]
    else:
        peaks = 1
        same_spot_value = 'undef'
        if params_single[2] > -0.372*(params_single[1]-min_layer_height) + 3.572:
            layer = 'disorganized'
            nuclear_peak = params_single[1]
        else:
            layer = 'organized'
            nuclear_peak = params_single[1]
            


    if (plot == True or save_name != False):
        fig, ax = plt.subplots()#(figsize =(3,2))
        ax.scatter(bins[:-1], counts_to_fit, label = 'Nuclear Position')
        ax.plot(bins_fit, fit_double, label = 'Double Gaussian Fit')
        ax.set_ylabel('Number of nuclei')
        ax.set_xlabel('Z Position ($\mu$m)')
        ax.set_xlim(0, 30)
        ax2=ax.twinx()
        ax2.plot(slice_heights, norm_intensities_actin, c='r')
        ax2.set_ylabel('Actin Intensity (A.U.)')
        ax2.axvline(x = min_layer_height, c='k', linestyle = '--', label = 'Layer Bounds')
        ax2.axvline(x = max_layer_height, c = 'k', linestyle = '--')
        print(min_layer_height, max_layer_height)#, params_single[1]-min_layer_height, params_single[2])
        fig.show
        if save_name != False:
            plt.savefig(save_name + '.pdf', bbox_inches = 'tight', pad_inches = 1)
            
    
    return nuclear_peak, layer

#Plots a scatter plot of terrain in the z-axis (top-down view).
##Size of each plotted circle is directly proportional to the volume of the cell it represents
###Darker color represents higher z value
def terrain_map(df, image, **kwargs):
    color = kwargs.get('color', 'magma_r')
    x_max, y_max, z_max, x_min, y_min, z_min, slice_heights = get_layer_position(df, image)
    df_cleared = clear_debris(df)
    save_name = kwargs.get('save_name', False)
    xs = df_cleared['x'].values - x_min
    ys = df_cleared['y'].values - y_min
    zs = df_cleared['z'].values - z_min
    vols = df_cleared['vol'].values
    cross_sections = np.pi*(np.cbrt(vols*3/(4*np.pi)))**2 #calculates size of circle from greatest cross section of cell
    fig, ax = plt.subplots(figsize=(5,5))
    ax.scatter(xs, -ys, s = cross_sections/2, c = zs, cmap = color, vmin = 0, vmax = 30)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim([0, x_max-x_min])
    ax.set_ylim([-(y_max-y_min), 0])
    if save_name != False:
        plt.savefig(save_name + '.pdf', bbox_inches = 'tight', pad_inches = 1)
    return

#Determines number of cells both above and inside the layer, and from this the percent of cells above the layer.
##Also returns cell density and layer classified based upon given criteria
def layer_determination(df, image, **kwargs):
    df_cleared = clear_debris(df)
    x_max, y_max, z_max, x_min, y_min, z_min, slice_heights = get_layer_position(df, image)
    min_layer_height, max_layer_height, layer_height, actin_peak, norm_intensities_actin = layer_height_actin(df, image)
    equivalence, num_peaks, p1, p2 = find_shoulders(df, image)
    nuclear_peak, layer = nuclei_distribution(df, image)
    
    actin_to_top = max_layer_height - actin_peak
    actin_to_bottom = actin_peak - min_layer_height


    peak_difference_rule = (actin_peak - nuclear_peak)>= 0
    actin_peak_position_rule = (actin_to_top - actin_to_bottom)<0
    deriv_peak_rule = num_peaks == 1 
    
    cells_above = np.sum(df_cleared['z']-z_min >= max_layer_height)
    cells_inside = np.sum(df_cleared['z']-z_min < max_layer_height)
    percentage_above = (cells_above/(cells_above+cells_inside)*100)
    cell_density = (cells_inside+cells_above) / (x_max-x_min)**2*1000
    

    if layer == 'disorganized':
        layer_classification = 'Disorganized'
        
    else:
        if actin_peak_position_rule and peak_difference_rule:
            if deriv_peak_rule or equivalence < 1:
                layer_classification = 'Intermediate' 
            else:
                layer_classification = 'Mature'
        else:
            layer_classification = 'Immature'
   
    return layer_classification, cells_above, cells_inside, percentage_above, cell_density

#Determines number of cells within the confines of the layer
def disorganization_extent(df, image, **kwargs):
    plot = kwargs.get('plot', False)
    save_name = kwargs.get('save_name', False)
    actin_channel = kwargs.get('actin_channel', 0)
    x_max, y_max, z_max, x_min, y_min, z_min, slice_heights = get_layer_position(df, image)
    df_cleared = clear_debris(df)
    min_layer_height, max_layer_height, layer_height, actin_peak, norm_intensities_actin = layer_height_actin(df, image)
    
    density = len(df_cleared)/(x_max-x_min)**2*1000
    if density <=2:
        bot_cutoff = 0.5
        top_cutoff = 0.6
    elif density>=6:
        bot_cutoff = 0.2
        top_cutoff = 0.8
    else:
        bot_cutoff = 0.5-0.3*(density-2)/4
        top_cutoff = 0.6+0.2*(density-2)/4
    print(density, bot_cutoff, top_cutoff)
    min_actin_slice = np.argwhere(norm_intensities_actin >= bot_cutoff)[0][0]
    max_actin_slice = min_actin_slice+15
    
    min_slice = slice_heights[min_actin_slice]
    max_slice = slice_heights[min_actin_slice]+8
    sum_proj_z = np.sum(image[min_actin_slice:max_actin_slice, actin_channel, :, :], axis = 0)
    df_clear_in_layer = df_cleared[df_cleared['z']>(min_slice+z_min)]
    df_clear_in_layer = df_clear_in_layer[df_clear_in_layer['z']<(max_slice+z_min)]
    real_xs = (df_clear_in_layer['x'] - x_min)*sum_proj_z.shape[0]/(x_max-x_min)
    real_ys = (df_clear_in_layer['y'] - y_min)*sum_proj_z.shape[1]/(y_max-y_min)
    
    basal_cells = len(df_clear_in_layer)
    
    if (plot == True):
        fig, ax = plt.subplots(figsize = (7,7))
        ax.imshow(sum_proj_z, cmap = 'Greys')
        ax.scatter(real_xs, real_ys, c = 'k')
        ax.set_xlim([0, sum_proj_z.shape[0]])
        ax.set_ylim([0, sum_proj_z.shape[1]])
        ax.set_xticks([])
        ax.set_yticks([])
        if save_name != False:
            plt.savefig(save_name + '.pdf', bbox_inches = 'tight', pad_inches = 1)
    
    return(basal_cells)


def batch_process(list_of_dfs, list_of_unshuffled_images, list_of_names, **kwargs):
    box_radius = kwargs.get('box_radius', False)
    save_name = kwargs.get('save_name', False)
    list_of_cells_above = []
    list_of_percent_above = []
    list_of_cells_in = []
    list_of_layer_class = []
    list_of_cell_densities = []
    list_of_layer_heights = []
    list_of_bad_images = []
    list_of_disorganization_extent = []
    list_of_equivalences = []
    list_of_widths = []
    list_of_peak_positions = []
    list_of_peak_numbers = []

    if box_radius != False:
        list_of_box_cells = []
        list_of_box_densities = []

    for i in range(len(list_of_dfs)):
        try:
            df = list_of_dfs[i]
            image = image_shuffle(list_of_unshuffled_images[i])
                  
            # Defining get_layer_position routine variables:
            num_z, num_c, num_x, num_y = image.shape
            df.columns = ['x', 'y', 'z', 'vol', 'positions']
            x_max = df['positions'][0]
            y_max = df['positions'][1]
            z_max = df['positions'][2]
            x_min = df['positions'][3]
            y_min = df['positions'][4]
            z_min = df['positions'][5]
            slice_heights = np.linspace(0, z_max-z_min, num = num_z)

            # Portion of clear_debris which calculates value of df_cleared
            vols = df['vol'].values 
            bottom_vol = np.mean(vols)- (1.5*np.std(vols)) #cutoff volume
            df_cleared = df[df['vol']>=bottom_vol] #all sources below the cutoff volume are excluded
            
            # Defining desired variables from layer_height_actin routine
            actin_channel = kwargs.get('actin_channel', 1)
            xy_proj_actin = np.sum(np.sum(image, axis = 3), axis = 2)[:,actin_channel].copy() #measuring the xy projecting of the actin intensity
                                                                                              # x and y were assigned to axis 2 and 3 in image shuffle
            norm_intensities_actin = (xy_proj_actin-np.min(xy_proj_actin))/np.max(xy_proj_actin-np.min(xy_proj_actin)) #normalize actin graph
            actin_peak = slice_heights[np.argwhere(norm_intensities_actin == np.max(norm_intensities_actin))][0][0] #determine actin peak z-value
            density = len(df_cleared)/(x_max-x_min)**2*1000
            if density <=2:
                bot_cutoff = 0.5
                top_cutoff = 0.6
            elif density>=6:
                bot_cutoff = 0.2
                top_cutoff = 0.8
            else:
                bot_cutoff = 0.5-0.3*(density-2)/4
                top_cutoff = 0.6+0.2*(density-2)/4
            min_actin_slice = np.argwhere(norm_intensities_actin >= bot_cutoff)[0][0]
            max_actin_slice = np.argwhere(norm_intensities_actin >= top_cutoff)[-1][0] #Determining locations of bottom and top of layer using empirical results
            min_layer_height = slice_heights[min_actin_slice]
            max_layer_height = slice_heights[max_actin_slice] #Determining values of layer height at bottom and top bounds found above
            layer_height = max_layer_height-min_layer_height
            actin_intensity_top = norm_intensities_actin[max_actin_slice] #actin intensity value at top of layer

            # Defining nuclei_distribution variables
            zs = df_cleared['z'].values - z_min
            bins = np.linspace(0, 50, 51)
            bins_fit = np.linspace(0, 50, 1000)
            counts, bins_aux = np.histogram(zs, bins = bins)
            counts_to_fit = smooth_array(counts)
            peak_height_guess = np.max(counts_to_fit)
            peak_loc_guess = bins[np.argwhere(counts_to_fit==peak_height_guess)[0][0]]
            p0_double = [peak_height_guess, peak_loc_guess, 2, 30, 2*peak_loc_guess, 5]
            p0_single = [peak_height_guess, peak_loc_guess, 2]
            bounds_double = ([0, 1, 1, 0, 1, 1], [400, 80, 15, 400, 80, 15])
            bounds_single = ([0, 1, 1], [400, 80, 15])
            peak_loc = np.argwhere(counts_to_fit==np.max(counts_to_fit))[0][0]
            params_double, params_covariance_double = optimize.curve_fit(double_gaussian_fit, bins[:-1], counts_to_fit, p0_double, bounds = bounds_double)
            params_single, params_covariance_single = optimize.curve_fit(gaussian_fit, bins[:-1], counts_to_fit, p0_single, bounds = bounds_single)
            fit_single = gaussian_fit(bins_fit, params_single[0], params_single[1], params_single[2])
            fit_double = double_gaussian_fit(bins_fit, params_double[0], params_double[1], params_double[2], params_double[3], params_double[4], params_double[5])

            mean_deviation = np.sqrt(np.mean((fit_single - fit_double)**2))

            if mean_deviation >5:
                peaks = 2
                if params_double[0]>=params_double[3]:
                    tall_peak = params_double[:3]
                    short_peak = params_double[3:]
                else:
                    tall_peak = params_double[3:] 
                    short_peak = params_double[:3]

                if params_double[1]<=params_double[4]:
                    left_peak = params_double[:3]
                    right_peak = params_double[3:]
                else:
                    left_peak = params_double[3:] 
                    right_peak = params_double[:3]

                a1, b1, c1 = left_peak
                a2, b2, c2 = right_peak

                A = 1/c2**2-1/c1**2
                B = 2*b1/c1**2-2*b2/c2**2
                C = (b2/c2)**2-(b1/c1)**2+np.log(a1/a2)

                roots = np.roots([A,B,C])
                intersection = np.max(gaussian_fit(roots, a1,b1,c1))
                same_spot_value = intersection / short_peak[0]

                if same_spot_value <= 0.5:
                    layer = 'organized'
                    nuclear_peak = left_peak[1]
                else:
                    layer = 'disorganized'
                    nuclear_peak = left_peak[1]
            else:
                peaks = 1
                same_spot_value = 'undef'
                if params_single[2] > -0.372*(params_single[1]-min_layer_height) + 3.572:
                    layer = 'disorganized'
                    nuclear_peak = params_single[1]
                else:
                    layer = 'organized'
                    nuclear_peak = params_single[1]
                    
            list_of_widths.append(params_single[2])
            list_of_peak_positions.append(params_single[1]-min_layer_height)
            list_of_peak_numbers.append(peaks)

            # Defining needed layer_determination and find_shoulder variables
            actin_profile_derivative = smooth_array(np.diff(norm_intensities_actin), window_size = 5)
            peaks = signal.find_peaks(actin_profile_derivative, prominence = 0.008)
            num_peaks = len(peaks[0])
            
            if len(peaks[0])==2:
                equivalence = peaks[1]['prominences'][1]/peaks[1]['prominences'][0]
            else:
                equivalence = 'undef'


            actin_to_top = max_layer_height - actin_peak
            actin_to_bottom = actin_peak - min_layer_height
            peak_difference_rule = (actin_peak - nuclear_peak)>= 0
            actin_peak_position_rule = (actin_to_top - actin_to_bottom)<0
            deriv_peak_rule = num_peaks == 1 

            cells_above = np.sum(df_cleared['z']-z_min >= max_layer_height)
            cells_inside = np.sum(df_cleared['z']-z_min < max_layer_height)
            percentage_above = (cells_above/(cells_above+cells_inside)*100)
            cell_density = (cells_inside+cells_above) / (x_max-x_min)**2*1000


            if layer == 'disorganized':
                layer_classification = 'Disorganized'

            else:
                if actin_peak_position_rule and peak_difference_rule:
                    if deriv_peak_rule or equivalence < 1:
                        layer_classification = 'Intermediate' 
                    else:
                        layer_classification = 'Mature'
                else:
                    layer_classification = 'Immature'

            # Defining disorganization_extent variables
            actin_channel = kwargs.get('actin_channel', 0)
            # We were not able to determine the maximum actin slice/height as before due to the amount of disorganized cells
            # above the layer which skew the actin intensity graph such that it's impossible to determine the top of the 
            # "layer" in disorganized cases. In the following line, we use empirical observations to manually determine 
            # the position of the top of the layer
            max_actin_slice_weird = min_actin_slice+15
            min_slice = slice_heights[min_actin_slice]
            max_slice = slice_heights[min_actin_slice]+8
            sum_proj_z = np.sum(image[min_actin_slice:max_actin_slice_weird, actin_channel, :, :], axis = 0)
            df_clear_in_layer = df_cleared[df_cleared['z']>(min_slice+z_min)]
            df_clear_in_layer = df_clear_in_layer[df_clear_in_layer['z']<(max_slice+z_min)]
            real_xs = (df_clear_in_layer['x'] - x_min)*sum_proj_z.shape[0]/(x_max-x_min)
            real_ys = (df_clear_in_layer['y'] - y_min)*sum_proj_z.shape[1]/(y_max-y_min)

            basal_cells = len(df_clear_in_layer)

            list_of_cells_above.append(cells_above)
            list_of_percent_above.append(percentage_above)
            list_of_cells_in.append(cells_inside)
            list_of_layer_class.append(layer_classification)
            list_of_cell_densities.append(cell_density)
            list_of_layer_heights.append(layer_height)
            list_of_disorganization_extent.append(basal_cells)
            list_of_equivalences.append(equivalence)

            if box_radius != False:
                cells_in_box, box_density = local_density(df, image, box_radius = box_radius)
                list_of_box_cells.append(cells_in_box)
                list_of_box_densities.append(box_density)
                
        except Exception as e: 
            print(e)
            list_of_bad_images.append(i)
            continue
    list_of_totals = []
    for i in range(len(list_of_cells_in)):
        list_of_totals.append(list_of_cells_in[i]+list_of_cells_above[i])
    if len(list_of_bad_images)>0:
        print('The following images would not complete processing: \n', list_of_bad_images)
        dict_list_of_names = np.delete(list_of_names, list_of_bad_images)
    else:
        dict_list_of_names = list_of_names

    if box_radius != False:
        dict_data = {'names': dict_list_of_names,
                     'Is this a layer?':list_of_layer_class,
                     'cells in layer': list_of_cells_in,
                     'cells above layer': list_of_cells_above,
                     'total number of cells': list_of_totals,
                     'disorganized cells in layer' : list_of_disorganization_extent,
                     'cell density': list_of_cell_densities,
                     'cells in box': list_of_box_cells,
                     'box density': list_of_box_densities,
                     'layer height': list_of_layer_heights,
                     '% above': list_of_percent_above,
                     'equivalence': list_of_equivalences,
                     'peaks': list_of_peak_numbers,
                     'widths': list_of_widths,
                     'positions': list_of_peak_positions
                    }
    else:
        dict_data = {'names': dict_list_of_names,
                     'Is this a layer?': list_of_layer_class,
                     'cells in layer': list_of_cells_in,
                     'cells above layer': list_of_cells_above,
                     'total number of cells': list_of_totals,
                     'disorganized cells in layer' : list_of_disorganization_extent,
                     'cell densities': list_of_cell_densities,
                     'layer height': list_of_layer_heights,
                     '% above': list_of_percent_above,
                     'equivalence': list_of_equivalences,
                     'peaks': list_of_peak_numbers,
                     'widths': list_of_widths,
                     'positions': list_of_peak_positions
                    }
    
    analyzed_df = pd.DataFrame(dict_data)
    if save_name != False:
        analyzed_df.to_csv(save_name + '.csv')
        return analyzed_df
    else:
        return analyzed_df

tic = time.time()
print(tic-toc)

0.003838777542114258


#### This final cell is an example which will output a spreadsheet with the qualitative and quantitative analyses of each image/segmentation input. 

In [44]:
toc = time.time()
batch_process(list_of_dfs, list_of_unshuffled_images, list_of_names, save_name = "name")
tic=time.time()
print(tic-toc)

0.2029421329498291
